# 퇴근후딴짓 작업형2유형

## (sample) 작업형 2유형

백화점 고객 1년간 구매데이터

성별 예측값(남자일 확률)을 csv파일로 생성
roc-auc평가지표에 따라 채점

In [ ]:
# 라이브러리
import pandas as pd

x_test = pd.read_csv("data/X_test.csv")
x_train = pd.read_csv("data/X_train.csv")
y_train = pd.read_csv("data/y_train.csv")

# EDA 데이터전처리, 피처링
print(x_train.isnull().sum())
x_train = x_train.fillna(0)
x_test = x_test.fillna(0)

x_train = x_train.drop(['cust_id'], axis =1)
cust_id = x_test.pop('cust_id')

# 범주형변수 피처 레이블인코딩
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
cols = x_train.select_dtypes('object').columns
for col in cols:
	x_train[col] = le.fit_transform(x_train[col])
	x_test[col] = le.fit_transform(x_test[col])

# 모델링 & 하이퍼파라미터 튜닝
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators = 100, max_depth = 5, random_state = 2022)
rf.fit(x_train, y_train['gender'])

pred = rf.predict_proba(x_test)

# 결과물 저장
output = pd.DataFrame({'cust_id': cust_id, 'gender': pred[:,1]})
output.to_csv('0000.csv', index = False)

## T2-1. 타이타닉 simple baseline 생존여부 예측모델 만들기
https://www.kaggle.com/code/agileteam/t2-1-titanic-simple-baseline/notebook


학습용 데이터 (X_train, y_train)을 이용하여 생존 예측 모형을 만든 후, 이를 평가용 데이터(X_test)에 적용하여 얻은 예측값을 다음과 같은 형식의 CSV파일로 생성하시오(제출한 모델의 성능은 accuracy 평가지표에 따라 채점)

In [ ]:
# 시험환경 세팅 (코드 변경 X)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def exam_data_load(df, target, id_name="", null_name=""):
    if id_name == "":
        df = df.reset_index().rename(columns={"index": "id"})
        id_name = 'id'
    else:
        id_name = id_name
    
    if null_name != "":
        df[df == null_name] = np.nan
    
    X_train, X_test = train_test_split(df, test_size=0.2, random_state=2021)
    
    y_train = X_train[[id_name, target]]
    X_train = X_train.drop(columns=[target])

    
    y_test = X_test[[id_name, target]]
    X_test = X_test.drop(columns=[target])
    return X_train, X_test, y_train, y_test 


In [ ]:
# 데이터 불러오기 
df = pd.read_csv("../input/titanic/train.csv")
X_train, X_test, y_train, y_test = exam_data_load(df, target='Survived', id_name='PassengerId')

In [ ]:
# EDA

X_train.shape, X_test.shape, y_train.shape, y_test.shape
X_train.head()
X_test.head()
y_train.head()
X_train.isnull().sum()
X_train.info()
# 생존비율
y_train.Survived.sum() / y_train.shape[0]

In [ ]:
# 전처리

# 불필요한 컬럼 제거
dropcols = ['PassengerId','Name','Ticket','Cabin']   
x_train = X_train.drop(columns = dropcols)
x_test = X_test.drop(columns = dropcols)
y = y_train["Survived"]

# 데이터타입 변경
x_train.Pclass = x_train.Pclass.astype("object")
x_test.Pclass = x_test.Pclass.astype("object")

# 더미변수화 (원핫인코딩)
x_tr_dum = pd.get_dummies(x_train, drop_first= True)
x_ts_dum = pd.get_dummies(x_test, drop_first = True)

# age변수 결측데이터 제거
x_tr_dum.dropna(axis = 0, inplace= True)
y = y[-x_tr_dum.Age.isnull()]
x_ts_dum.dropna(axis = 0, inplace= True)
x_ts_dum.Age = x_ts_dum.Age.astype("int64")

In [ ]:
# 모델링 및 평가
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators = 200, max_depth = 7, random_state = 2021)
rf.fit(x_tr_dum, y)

pred = rf.predict(x_ts_dum)
rf.score(x_tr_dum, y)

# 결과 
output = pd.DataFrame({'PassengerId': X_test.PassengerId, 'Survived': predictions})
output.head()